<a href="https://colab.research.google.com/github/Abyy98/W7T5/blob/main/YOLO_Practical_Exam5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install opencv-python

!pip install ultralytics

!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 17.5 MB/s eta 0:00:00


In [ ]:
pip install --upgrade torch torchvision opencv-python ultralytics


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [27]:
import cv2
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)


## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [28]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video Height: {height}, Width: {width}, FPS: {fps}")


Video Height: 1080, Width: 1920, FPS: 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [29]:
output_path = 'output5_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [30]:

roi_top = (width // 2 - 100, height // 4 - 20, width // 2 + 100, height // 4 + 70)
roi_bottom = (width // 2 - 50, 3 * height // 4 - 80, width // 2 + 150, 3 * height // 4 + 20)
roi_right = (3 * width // 4 - 120, height // 2 - 70, 3 * width // 4 + 30, height // 2 + 70)
roi_left = (width // 4 - 20, height // 2 - 100, width // 4 + 70, height // 2 + 100)



## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [35]:
count_top = 0
count_bottom = 0
count_right = 0
count_left = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    cv2.rectangle(frame, (roi_top[0], roi_top[1]), (roi_top[2], roi_top[3]), (0, 255, 0), 2)
    cv2.rectangle(frame, (roi_bottom[0], roi_bottom[1]), (roi_bottom[2], roi_bottom[3]), (0, 255, 255), 2)
    cv2.rectangle(frame, (roi_right[0], roi_right[1]), (roi_right[2], roi_right[3]), (255, 105, 180), 2)
    cv2.rectangle(frame, (roi_left[0], roi_left[1]), (roi_left[2], roi_left[3]), (255, 0, 0), 2)

    for result in results:
        if hasattr(result, 'boxes'):
            detections = result.boxes.xyxy.cpu().numpy()
            for box in detections:
                x1, y1, x2, y2 = box

                if roi_top[0] < x1 < roi_top[2] and roi_top[1] < y1 < roi_top[3]:
                    count_top += 1
                elif roi_bottom[0] < x1 < roi_bottom[2] and roi_bottom[1] < y1 < roi_bottom[3]:
                    count_bottom += 1
                elif roi_right[0] < x1 < roi_right[2] and roi_right[1] < y1 < roi_right[3]:
                    count_right += 1
                elif roi_left[0] < x1 < roi_left[2] and roi_left[1] < y1 < roi_left[3]:
                    count_left += 1
    cv2.putText(frame, f'Top: {count_top}', (roi_top[0], roi_top[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(frame, f'Bottom: {count_bottom}', (roi_bottom[0], roi_bottom[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
    cv2.putText(frame, f'Right: {count_right}', (roi_right[0], roi_right[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 105, 180), 2)
    cv2.putText(frame, f'Left: {count_left}', (roi_left[0], roi_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.putText(frame, 'Abeera', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 61.6ms
Speed: 5.3ms preprocess, 61.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 61.5ms
Speed: 3.0ms preprocess, 61.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 61.0ms
Speed: 3.3ms preprocess, 61.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 57.8ms
Speed: 3.1ms preprocess, 57.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 43.6ms
Speed: 3.2ms preprocess, 43.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 42.1ms
Speed: 3.8ms preprocess, 42.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 41.3ms
Speed: 3.2ms preprocess, 41.3ms inferenc

## Save and Submit Your Work

Submit both the notebook and the output video

1-Frist Loaded the YOLO Model

2-Used OpenCV to reed the video file and procesed it frame by frame. For each frame, I detect vehicles and drew bounding boxes around the detected obects.

3-I defined ares in the video frame that represent the exits on the top, bottom,left, and right sides. This make trakcing when vehicles moved through these area

4-For each frame, trackd the movement of the detected vehicles and counted how many exited through each defined region.



5-displayed the bounding boxes, vehicle counts, and other relevant informaton on the video frames.This included adding my name 'Abeer' at the top of the video and showing the count of vehicles exiting from each side.